In [1]:
# syft absolute
import syft as sy
from syft import ActionObject
from syft.client.syncing import compare_states
from syft.client.syncing import resolve

In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=4fab214805b9466b99b4a024198e17d4 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=2d6f3e86ad7640dfb7396c7a1850cbe7 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
client_low.register(
    email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw"
)
client_low_ds = node_low.login(email="newuser@openmined.org", password="pw")

Logged into <test_l: Low side Domain> as <newuser@openmined.org>


# create datasets

In [5]:
# third party
import numpy as np

In [6]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 13.63it/s]

Uploading: numpy-data


SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [7]:
mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
# private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    id=dataset_high.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=ActionObject.empty(data_node_id=client_high.id),
            shape=mock_low.shape,
            mock_is_real=True,
        )
    ],
)

res = client_low.upload_dataset(dataset_low)

  0%|                                                                                             | 0/1 [00:00<?, ?it/s]

Uploading: numpy-data


100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 30.13it/s]


In [8]:
dataset_storage_permissions = node_low.python_node.get_service(
    "datasetservice"
).stash.partition.storage_permissions
dataset_id = client_low.datasets[0].id

assert dataset_storage_permissions[dataset_id] == {node_low.python_node.id}

# Data Scientist: make requests

In [9]:
@sy.syft_function()
def sub_function(data) -> float:
    print("executing sub function")
    return data.mean()

client_low_ds.code.submit(sub_function)

SyftSuccess: Syft function 'sub_function' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftSuccess: User Code Submitted

In [10]:
data_low = client_low_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(domain, data) -> float:
    print("executing main function")
    job = domain.launch_job(sub_function, data=data)
    res = job.wait().get() * 100
    return res

res = compute_mean(data=data_low.mock, blocking=False)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
SyftInfo: Closing the node after time_alive=300 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/605247446bc94c9faee61eeaf5f129f1.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=2 image_uid=b68f9f1101cc4d6087bc0721b031ea4d in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_1602: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Approving request for domain ephemeral_node_compute_mean_1602


In [11]:
res.wait()

14/03/24 22:42:46 FUNCTION LOG (f0fb2d5d33f44e19bb5eceb0d26f44d6): executing main function
14/03/24 22:42:46 EXCEPTION LOG (f0fb2d5d33f44e19bb5eceb0d26f44d6):

Encountered while executing compute_mean:
Traceback (most recent call last):
  File "/Users/eelco/dev/PySyft/packages/syft/src/syft/service/code/user_code.py", line 1461, in execute_byte_code
    result = eval(evil_string, _globals, _locals)  # nosec
  File "<string>", line 1, in <module>
  File "<string>", line 8, in user_func_compute_mean_9c58f0c1d9439eed22e249736f0e9ea757423ef3afa62fcedb06777c584fe714_d25fd5e947bf5697d50918b4581ff279cdc3e3ac3f30a82d6b9bb399bac86edf
  File "<string>", line 6, in compute_mean
  File "/Users/eelco/dev/PySyft/packages/syft/src/syft/service/job/job_stash.py", line 436, in wait
    raise ValueError(
ValueError: Can't access Syft API. You must login to 605247446bc94c9faee61eeaf5f129f1




AttributeError: 'Err' object has no attribute 'id'

In [12]:
client_low_ds.code.request_code_execution(compute_mean)

syft.service.request.request.Request

In [13]:
client_low_ds.code.compute_mean(data=data_low)

SyftError: Execution denied, Please wait for the code to be approved

## Data Owner: Sync to high side

In [14]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

[]
[]


In [15]:
low_state

syft.service.sync.sync_state.SyncState

In [16]:
assert (
    set(low_state.objects.keys())
    == set(low_state.permissions.keys())
    == set(low_state.storage_permissions.keys())
)

In [17]:
diff_state = compare_states(low_state, high_state)

In [18]:
diff_state

syft.service.sync.diff_state.NodeDiff

In [19]:
diff_state.hierarchies[0].diffs

[ObjectDiff[UserCodeStatusCollection](#f0516e5acf4240bdb476cc4ccfead6a1),
 ObjectDiff[UserCode](#818e2dcdb91f4364b0117bfce6764252),
 ObjectDiff[UserCode](#524204e5c0ed4005808be7fb2cc014f3)]

In [20]:
resolved_state_low, resolved_state_high = resolve(diff_state, decision="low")

LOW SIDE STATE:

―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #818e2dcdb91f4364b0117bfce6764252:
service_func_name: compute_mean
input_owners: [
‎ ‎ ‎ ‎ test_l
]
code_status: [
‎ ‎ ‎ ‎ Node: test_l, Status: pending
]
worker_pool_name: default-pool

    ―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODESTATUSCOLLECTION #f0516e5acf4240bdb476cc4ccfead6a1:
    approved: False
    status_dict: {
    ‎ ‎ ‎ ‎ node_id=<UID: 8a1c04544655402190588aec30079bc3>
        verify_key=1c0590b894d19530970da6bf09bb302afa3e473a4cccdacbace0e20223ee3367
        node_name='test_l': (<UserCodeStatus.PENDING: 'pending'>, '')
    }

    ―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODE #524204e5c0ed4005808be7fb2cc014f3:
    service_func_name: sub_function
    input_owners: [
    ]
    code_status: [
    ‎ ‎ ‎ ‎ Node: test_l, St

In [21]:
print("Resolved state low side")
print(resolved_state_low)
print()
print("Resolved state high side")
print(resolved_state_high)

Resolved state low side
ResolvedSyncState(
  create_objs=[],
  update_objs=[],
  delete_objs=[]
  new_permissions=[]
)

Resolved state high side
ResolvedSyncState(
  create_objs=[{NodeIdentity <name=test_l, id=8a1c0454, 🔑=1c0590b8>: (<UserCodeStatus.PENDING: 'pending'>, '')}, syft.service.code.user_code.UserCode, syft.service.code.user_code.UserCode, syft.service.request.request.Request, {NodeIdentity <name=test_l, id=8a1c0454, 🔑=1c0590b8>: (<UserCodeStatus.PENDING: 'pending'>, '')}],
  update_objs=[],
  delete_objs=[]
  new_permissions=[]
)


In [22]:
assert len(resolved_state_high.new_permissions) == 0

new_objs = resolved_state_high.create_objs
assert {o.id for o in new_objs} == {
    sp.uid for sp in resolved_state_high.new_storage_permissions
}

In [23]:
client_low.apply_state(resolved_state_low)

[]


SyftSuccess: Synced 0 items

In [24]:
client_high.apply_state(resolved_state_high)

[]


SyftSuccess: Synced 5 items

In [25]:
request_storage_permissions = node_high.python_node.get_service(
    "requestservice"
).stash.partition.storage_permissions
request_id = client_high.requests[0].id
node_high_id = node_high.python_node.id

assert node_high_id in request_storage_permissions[request_id]

# Data Owner: Run code high-side

In [26]:
data_high = client_high.datasets[0].assets[0]

In [27]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)
display(job_high)

```python
class Job:
    id: UID = d76d16a7bf3341deb0caccd402bb4fd5
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [28]:
# wait for the result
job_high.wait().get()

SyftInfo: Node Landed!


14/03/24 22:34:25 FUNCTION LOG (d76d16a7bf3341deb0caccd402bb4fd5): executing main function
14/03/24 22:34:27 FUNCTION LOG (321fe1bb060a415881ab3d292b3ef908): executing sub function


17.0

In [29]:
job_info = job_high.info(public_metadata=True, result=True)

request = client_high.requests[0]
result_obj = job_high.result

In [30]:
job_info

syft.service.job.job_stash.JobInfo

In [31]:
# syft absolute
from syft import SyftError
from syft import SyftSuccess

# Accepting the result directly gives an error
accept_res = request.accept_by_depositing_result(result_obj)
assert isinstance(accept_res, SyftError)

In [32]:
accept_res = request.accept_by_depositing_result(job_info)

assert isinstance(accept_res, SyftSuccess)
accept_res

Approving request for domain test_h
returning existing job
setting permission
None
None
Approving request for domain test_h
ADDING PERMISSION [READ: 07912a32e4354bb39cb2aee9a56eb4b9 as 6468dc67bbf0336195fd84d19cd517a170025956dc5cac95e9eb933e7a27f5b3] 07912a32e4354bb39cb2aee9a56eb4b9
Job(d76d16a7bf3341deb0caccd402bb4fd5) Setting new result 07912a32e4354bb39cb2aee9a56eb4b9 -> 07912a32e4354bb39cb2aee9a56eb4b9


SyftSuccess: Request submitted for updating result.

In [33]:
# Need to refresh Job because we overwrite result
job_high = client_high.code.compute_mean.jobs[0]

action_store_high = node_high.python_node.get_service("actionservice").store
blob_store_high = node_high.python_node.get_service(
    "blobstorageservice"
).stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_high.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_high.permissions[job_high.result.syft_blob_storage_entry_id]
)

In [34]:
assert action_store_high.storage_permissions[job_high.result.id.id] == {
    node_high.python_node.id
}

## Data Owner: Sync back to low-side

In [35]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

if isinstance(low_state, sy.SyftError):
    print("low")
    print(low_state.message)

if isinstance(high_state, sy.SyftError):
    print("high")
    print(high_state.message)

[]
[Pointer:
```python
class ActionDataLink:
  id: str = 6b96905e1d8a4c6299938035a38af216

```, Pointer:
17.0]


In [41]:
list(high_state.objects.values())[-2]

```python
Pointer
```
```python
class ActionDataLink:
  id: str = 6b96905e1d8a4c6299938035a38af216

```

In [54]:
diff_state_2 = compare_states(low_state, high_state)

In [55]:
resolved_state_low, resolved_state_high = resolve(diff_state_2, share_private_objects=True)

LOW SIDE STATE:

―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #818e2dcdb91f4364b0117bfce6764252:
service_func_name: compute_mean
input_owners: [
‎ ‎ ‎ ‎ test_l
]
code_status: [
‎ ‎ ‎ ‎ Node: test_l, Status: pending
]

    ―――― DIFF ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODESTATUSCOLLECTION #f0516e5acf4240bdb476cc4ccfead6a1:
    status_dict: (<UserCodeStatus.PENDING: 'pending'>, '')

    ―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODE #524204e5c0ed4005808be7fb2cc014f3:
    service_func_name: sub_function
    input_owners: [
    ]
    code_status: [
    ‎ ‎ ‎ ‎ Node: test_l, Status: pending
    ]



HIGH SIDE STATE:

―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #818e2dcdb91f4364b0117bfce6764252:
service_func_name: compute_mean
input_owners

 high


Decision: Syncing 3 objects from high side


LOW SIDE STATE:

―――― DIFF ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

REQUEST #9f79b07b1e34417f8597fb3ad8fd8b5e:
node_uid: 8a1c04544655402190588aec30079bc3
changes: [
‎ ‎ ‎ ‎ syft.service.request.request.UserCodeStatusChange
]
history: [
]

    ―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODE #818e2dcdb91f4364b0117bfce6764252:
    service_func_name: compute_mean
    input_owners: [
    ‎ ‎ ‎ ‎ test_l
    ]
    code_status: [
    ‎ ‎ ‎ ‎ Node: test_l, Status: pending
    ]

        ―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

        USERCODE #524204e5c0ed4005808be7fb2cc014f3:
        service_func_name: sub_function
        input_owners: [
        ]
        code_status: [
        ‎ ‎ ‎ ‎ Node: test_l, Status: pending
        ]



HIGH SIDE STATE:

―――― DIFF ――――――――――――――――――――――――――――

 high


Decision: Syncing 3 objects from high side


LOW SIDE STATE:

―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #818e2dcdb91f4364b0117bfce6764252:
service_func_name: compute_mean
input_owners: [
‎ ‎ ‎ ‎ test_l
]
code_status: [
‎ ‎ ‎ ‎ Node: test_l, Status: pending
]

    ―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODE #524204e5c0ed4005808be7fb2cc014f3:
    service_func_name: sub_function
    input_owners: [
    ]
    code_status: [
    ‎ ‎ ‎ ‎ Node: test_l, Status: pending
    ]



HIGH SIDE STATE:

―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #818e2dcdb91f4364b0117bfce6764252:
service_func_name: compute_mean
input_owners: [
‎ ‎ ‎ ‎ test_l
]
code_status: [
‎ ‎ ‎ ‎ Node: test_h, Status: approved
]

    ―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

 

 high


Decision: Syncing 9 objects from high side




In [56]:
print(resolved_state_low)
print()
print(resolved_state_high)

ResolvedSyncState(
  create_objs=[syft.service.output.output_service.ExecutionOutput, syft.service.job.job_stash.Job, syft.service.job.job_stash.Job, Pointer:
17.0, syft.service.log.log.SyftLog, syft.service.log.log.SyftLog, Pointer:
syft.service.action.action_data_empty.ActionDataLink],
  update_objs=[{NodeIdentity <name=test_h, id=8212e679, 🔑=d311a667>: (<UserCodeStatus.APPROVED: 'approved'>, '')}, syft.service.request.request.Request],
  delete_objs=[]
  new_permissions=[[READ: d76d16a7bf3341deb0caccd402bb4fd5 as 6468dc67bbf0336195fd84d19cd517a170025956dc5cac95e9eb933e7a27f5b3], [READ: 321fe1bb060a415881ab3d292b3ef908 as 6468dc67bbf0336195fd84d19cd517a170025956dc5cac95e9eb933e7a27f5b3], [READ: 3ec1d8c326624fc78b18882444708fde as 6468dc67bbf0336195fd84d19cd517a170025956dc5cac95e9eb933e7a27f5b3], [READ: bfec41d2cc8d4804978978d4c87368e6 as 6468dc67bbf0336195fd84d19cd517a170025956dc5cac95e9eb933e7a27f5b3], [READ: 8ad86aa308364ea2a98ce8cb4341093f as 6468dc67bbf0336195fd84d19cd517a1700259

In [57]:
res = client_low.apply_state(resolved_state_low)
res

[Pointer:
```python
class ActionDataLink:
  id: str = 6b96905e1d8a4c6299938035a38af216

```, Pointer:
17.0]


SyftSuccess: Synced 9 items

In [58]:
client_high.apply_state(resolved_state_high)

[Pointer:
```python
class ActionDataLink:
  id: str = 6b96905e1d8a4c6299938035a38af216

```, Pointer:
17.0]


SyftSuccess: Synced 0 items

In [59]:
action_store_low = node_low.python_node.get_service("actionservice").store
blob_store_low = node_low.python_node.get_service("blobstorageservice").stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_low.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_low.permissions[job_high.result.syft_blob_storage_entry_id]
)

In [60]:
result_storage_permissions = node_low.python_node.get_service(
    "actionservice"
).store.storage_permissions[job_high.result.id.id]
assert len(result_storage_permissions) == 1

log_storage_permissions = node_low.python_node.get_service(
    "logservice"
).stash.partition.storage_permissions[job_high.log_id]
assert len(log_storage_permissions) == 1

# Run code low

## Run

In [62]:
client_low_ds._fetch_api(client_low.credentials)

In [67]:
client_low_ds.code.compute_mean.jobs[0].result.get()

17.0

In [68]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low

```python
Pointer
```
17.0

In [69]:
res_low.id == job_high.result.id

False

In [70]:
code = client_low_ds.code[0]

assert res_low.get() == private_high.mean()
assert (
    res_low.id.id
    == job_high.result.id.id
    == code.output_history[-1].output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

AssertionError: 

In [71]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low

```python
Pointer
```
17.0

In [72]:
code = client_low_ds.code[0]

assert res_low.get() == private_high.mean()
assert (
    res_low.id.id
    == job_high.result.id.id
    == code.output_history[-1].output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

AssertionError: 

In [73]:
private_high.mean()

17.0

In [74]:
job_low = client_low_ds.code.compute_mean(data=data_low, blocking=False)
job_low

SyftWarning: There are existing jobs for this user code, returning the latest one

```python
class Job:
    id: UID = d76d16a7bf3341deb0caccd402bb4fd5
    status: completed
    has_parent: False
    result: ActionDataEmpty <None>
    logs:

0 Log 8ad86aa308364ea2a98ce8cb4341093f not available
JOB COMPLETED
    
```

In [75]:
job_low.wait().get()

17.0

In [76]:
job_low.logs()

Log 8ad86aa308364ea2a98ce8cb4341093f not available


In [77]:
assert job_low.id == job_high.id
assert job_low.result.id == job_high.result.id
assert (
    job_low.result.syft_blob_storage_entry_id
    == job_high.result.syft_blob_storage_entry_id
)